# Datenanalyse


---

Datensatz: [Synthea Breast Cancer Dataset](https://github.com/Fuenfgeld/DMA2023TeamA/tree/main/Daten/Quelldaten)

Primär- und Fremdschlüsseldefinitionen: [Synthea GitHub Repository](https://github.com/synthetichealth/synthea/wiki/CSV-File-Data-Dictionary)

Projektgruppe GitHub Repository: [DMA2023TeamA](https://github.com/Fuenfgeld/DMA2023TeamA)

Source-DB: [GoogleDrive Ablage](https://drive.google.com/drive/folders/1k5cfjGXjNHmwQkydzjTdVHoBvCniBU_W), erstellt mit [Setup_and_fill_Database.ipynb](https://github.com/Fuenfgeld/DMA2023TeamA/blob/main/Code/Setup_and_fill_Database.ipynb)

Data Warehouse-Datenbank: [GoogleDrive Ablage](https://drive.google.com/file/d/1l-HcqCezubHnR737DkbiRzdHanP7_g_D), erstellt mit [ETL_process.ipynb](https://github.com/Fuenfgeld/DMA2023TeamA/blob/main/Code/ETL_Process.ipynb)


*Version*: 3.0

Version Date: 17/02/2023

Changes: 

**0.2**
* Erstellung eines Analysedatensatzes aus den aus der DWH-Datenbank extrahierten Daten (Gesamtkosten für Aufenthalte / Prozeduren / Medikationen pro Patient)
* Ergänzung Versionsnummern für neue Pakete

**1.0**
* Explorative Datenanalyse (EDA)
* Machine Learning Ansätze

**1.1**
* ergänzende Dokumentation der Analyse im Skript

**2.0**
* Installation einer neuen Version von Matplotlib
* zusätzliche Plots (Geschlecht und ethnischer Hintergrund nach Diagnose)
* Überarbeitung der Plots (Beschriftungen, Titel)
* Anpassung Analysen: NaN -> 0 für Berechnungen, Einbeziehung Prozedurkosten

**2.1**
* Anpassung der Pfade
* Speicherung ausgeleitete / verarbeitete Datentabellen als csv-Dateien
* Speicherung erstellte Plots (EDA) als png-Dateien

**3.0**
* Ergänzung von Hashfunktionen und Checksummen um Daten zu verifizieren
* Dokumentation im Machine Learning Teil ergänzt

# Vorbereitung des Notebooks

wichtig: das Notebook erfordert eine neuere Version von Matplotlib als standardmäßig in GoogleColab vorhanden. Zu Beginn muss daher die erste Codezeile ausgeführt werden, um die aktuellste Version zu installieren. Im Anschluss ist ein Neustart der Runtime erforderlich, um den Rest des Notebooks auszuführen.

In [ ]:
# Installation der aktuellsten Version von Matplotlib (erfordert Neustart der Runtime)
!pip install matplotlib --upgrade

In [ ]:
# Vorsichtshalber: Löschen aller Variablen
%reset -f

# Laden der benötigten Libraries
from google.colab import drive
import os
import sqlite3 as sq
from sqlite3 import Error
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport
from pandas.util import hash_pandas_object

# Ausgabe der Plots innerhalb des Notebooks
%matplotlib inline

In [ ]:
# Google Drive mounten, force_remount auf True setzen, damit ein Remount erzwungen wird
drive.mount('/content/gdrive/', force_remount=True)

# Pfad zum Projektverzeichnis setzen und dorthin wechseln
PROJECT_PATH = "/content/gdrive/Shareddrives/DMA_Datenprojekt_TeamA/"
os.chdir(PROJECT_PATH)

# Datenbankordner auf dem Shareddrive checken, es müssen source_breast_cancer.db und DWH_breast_cancer.db vorhanden sein
!ls "/content/gdrive/Shareddrives/DMA_Datenprojekt_TeamA/Daten/Datenbank"

# Patiententyp festlegen
patient_type = "breast_cancer"

# Pfad zur DWH-Datenbank setzen
DB_DWH_PATH = os.path.join(PROJECT_PATH, "Daten", "Datenbank", "DWH_breast_cancer.db")

# Pfad zum Analyse-Datensatz setzen
DATA_PATH = os.path.join(PROJECT_PATH, "Daten", "Analyse")

# Pfad zum Ergebnisordner setzen
RESULTS_PATH = os.path.join(PROJECT_PATH, "Ergebnisse")

# Check
print("\n" + PROJECT_PATH)
print("\n" + DB_DWH_PATH)
print("\n" + DATA_PATH)
print("\n" + RESULTS_PATH)

# Versionen der verwendeten Pakete abfragen 

Die Versionen der verwendeten Python-Installation und der Python-Pakete abfragen. 

In [ ]:
# Python-Version
print("Python-Version:")
!python --version

# Pandas-Version
print("\n" + "Pandas-Version:")
print("Pandas " + pd.__version__)

# numpy-Version
print("\n" + "numpy-Version:")
print("numpy " + np.version.version)

# sqlite-Version
print("\n" + "sqlite3-Version:")
print("sqlite3 " + sq.sqlite_version)

# Seaborn-Version
print("\n" + "Seaborn-Version:")
print("Seaborn " + sns.__version__)

# Matplotlib-Version
print("\n" + "Matplotlib-Version:")
print("Matplotlib " + matplotlib.__version__)

# sklearn-Version
print("\n" + "sklearn-Version:")
print("sklearn " + matplotlib.__version__)


# Extraktion der Daten aus der DWH-Datenbank


In [ ]:
# Datenbankverbindung zum DWH aufbauen
dwh_conn = sq.connect(DB_DWH_PATH) 
if dwh_conn is not None:
  dwh_cursor = dwh_conn.cursor()
else:
  print("Verbindung fehlgeschlagen. Bitte überprüfen!")

# Alle Tabellennamen aus der Datenbank ziehen
dwh_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablelist = dwh_cursor.fetchall()
tablelist

## Ausleitung der Daten zu den Behandlungskosten

In [ ]:
# SQL Abfrage zur Ausleitung der Daten zu den Behandlungskosten
extract_costs = """SELECT enc.enc_id,
                enc.patient,
                pat.gender    AS PAT_GENDER,
                pat.race      AS PAT_RACE,
                enc.enc_payer,
                pay.NAME      AS ENC_PAYER_NAME,
                enc.enc_base_cost,
                enc.enc_total_claim_cost,
                enc.enc_payer_coverage,
                enc.pro_code,
                enc.pro_base_cost,
                enc.med_code,
                enc.med_base_cost,
                enc.med_dispenses,
                enc.med_total_cost,
                enc.med_payer_coverage,
                enc.med_payer,
                enc.con_code,
                sct.term      AS CON_TERM
                FROM  f_encounter_costs AS enc
                      JOIN d_patients AS pat
                        ON enc.patient = pat.id
                      JOIN d_payers AS pay
                        ON enc.enc_payer = pay.id
                      LEFT JOIN d_snomedct AS sct
                        ON enc.con_code = sct.code
                ORDER  BY patient, enc.enc_id;"""
data_costs = pd.read_sql(extract_costs, dwh_conn)
data_costs.head()

In [ ]:
# ausgeleitete Daten in CSV Datei schreiben dwh_data_costs.csv
with open(os.path.join(DATA_PATH, "dwh_data_costs.csv"), 'w', encoding = 'utf-8') as f:
  data_costs.to_csv(f, index=False)

In [ ]:
# Hashwerte für ausgeleitete Daten zu den Behandlungskosten berechnen (ohne Spalte PATIENT)
data_costshashes = hash_pandas_object(data_costs.drop(columns=['PATIENT']))

In [ ]:
# Checksumme der Hashwerte berechnen
data_costshashes.sum()

In [ ]:
# Vergleich der berechneten Checksumme mit dem zu erwarteten Wert (5372785863725780577)
if data_costshashes.sum() == 5372785863725780577:
  print("Die Überprüfung der ausgeleiteten Daten zu den Behandlungskosten mittels Checksumme war erfolgreich. Die Checksumme stimmt mit dem im Projekt verwendeten Wert überein.")
else:
  print("Die Überprüfung der ausgeleiteten Daten zu den Behandlungskosten mittels Checksumme war nicht erfolgreich. Es liegen Abweichungen zu den im Projekt verwendeten Daten vor.")

## Ausleitung der Daten zur Diagnosegruppe

In [ ]:
# SQL Abfrage zur Ausleitung der Diagnoseart (none = keine, breast_cancer = Brustkrebs, other = andere Diagnose)
extract_diagnoses = """SELECT pat.id AS patient,
                              CASE
                                WHEN pat.id IN (SELECT DISTINCT( enc.patient )
                                                FROM   f_encounter_costs AS enc
                                                WHERE  enc.con_code = "254837009") THEN 'breast_cancer'
                                WHEN pat.id IN (SELECT DISTINCT( enc.patient )
                                                FROM   f_encounter_costs AS enc
                                                WHERE  NOT enc.con_code = "254837009"
                                                        AND enc.con_code IS NOT NULL) THEN 'other'
                                ELSE 'none'
                              END    AS diagnosis
                        FROM   d_patients AS pat;"""
data_diagnoses = pd.read_sql(extract_diagnoses, dwh_conn)
data_diagnoses.head()

In [ ]:
# ausgeleitete Daten in CSV Datei schreiben dwh_data_diagnoses.csv
with open(os.path.join(DATA_PATH, "dwh_data_diagnoses.csv"), 'w', encoding = 'utf-8') as f:
  data_diagnoses.to_csv(f, index=False)

In [ ]:
# Hashwerte für ausgeleitete Daten zur Diagnosegruppe berechnen (ohne Spalte patient)
data_diagnoseshashes = hash_pandas_object(data_diagnoses.drop(columns=['patient']))

In [ ]:
# Checksumme der Hashwerte berechnen
data_diagnoseshashes.sum()

In [ ]:
# Vergleich der berechneten Checksumme mit dem zu erwarteten Wert (3169026638659937830)
if data_diagnoseshashes.sum() == 3169026638659937830:
  print("Die Überprüfung der ausgeleiteten Daten zur Diagnosegruppe mittels Checksumme war erfolgreich. Die Checksumme stimmt mit dem im Projekt verwendeten Wert überein.")
else:
  print("Die Überprüfung der ausgeleiteten Daten zur Diagnosegruppe mittels Checksumme war nicht erfolgreich. Es liegen Abweichungen zu den im Projekt verwendeten Daten vor.")

In [ ]:
# Commit und Close
dwh_conn.commit()
dwh_conn.close()

# Erstellung einer Analysetabelle



## Explorative Datenanalyse der aus der DWH-Datenbank erstellten Tabellen

In [ ]:
# Überblick über Tabelle data_costs
data_costs.info()

In [ ]:
# Anzahl einmaliger Werte pro Spalte in Tabelle data_costs
data_costs.nunique(axis=0)

In [ ]:
# Analyse Duplikate in Tabelle data_costs (je nach Kostentyp)
print("Anzahl doppelte Aufenthalte:", data_costs.duplicated(subset=["PATIENT", "ENC_ID", "ENC_BASE_COST", "ENC_PAYER_COVERAGE"]).sum())
print("Anzahl doppelte Prozeduren:", data_costs.duplicated(subset=["PATIENT", "ENC_ID", "PRO_CODE", "PRO_BASE_COST"]).sum())
print("Anzahl doppelte Medikationen:", data_costs.duplicated(subset=["PATIENT", "ENC_ID", "MED_CODE", "MED_BASE_COST", "MED_DISPENSES", "MED_TOTAL_COST", "MED_PAYER_COVERAGE"]).sum())

In [ ]:
# Überblick über Tabelle data_diagnoses
data_diagnoses.info()

In [ ]:
# Anzahl einmaliger Werte pro Spalte in Tabelle data_diagnoses
data_diagnoses.nunique(axis=0)

In [ ]:
# Analyse Duplikate in Tabelle data_diagnoses
print("Anzahl doppelte Patienten:", data_costs.duplicated().sum())

## Erstellung Tabelle mit Gesamtkosten für Aufenthalte pro Patient

In [ ]:
# Erstellung Subset mit Kostendaten zu Aufenthalten
data_encounters = data_costs[["PATIENT", "ENC_ID", "ENC_BASE_COST", "ENC_PAYER_COVERAGE"]]
data_encounters.shape

In [ ]:
# Entfernen von Duplikaten (gleicher Patient, gleicher Aufenthalt, gleiche Kosten)
data_encounters_nodups = data_encounters.drop_duplicates()
data_encounters_nodups.shape
print(len(data_encounters) - len(data_encounters_nodups), "Duplikate entfernt.")

In [ ]:
# Gruppierung nach Patienten, Aufsummierung der Kosten
data_encounters_grouped = data_encounters_nodups.groupby(by="PATIENT")
data_encounters_final = data_encounters_grouped[["ENC_BASE_COST", "ENC_PAYER_COVERAGE"]].sum()
data_encounters_final.rename(columns={"ENC_BASE_COST": "OVERALL_ENC_BASE_COST", 
                                        "ENC_PAYER_COVERAGE": "OVERALL_ENC_PAYER_COVERAGE"}, inplace=True)
data_encounters_final["NUMBER_ENCOUNTERS"] = data_encounters_grouped.size()
data_encounters_final.shape

In [ ]:
data_encounters_final

## Erstellung Tabelle mit Gesamtkosten für Prozeduren pro Patient

In [ ]:
# Erstellung Subset mit Kostendaten zu Prozeduren
data_procedures = data_costs[["PATIENT", "ENC_ID", "PRO_CODE", "PRO_BASE_COST"]]
data_procedures.shape

In [ ]:
# Entfernen von Duplikaten (gleicher Patient, gleicher Aufenthalt, gleiche Prozedur, gleiche Kosten)
data_procedures_nodups = data_procedures.drop_duplicates()
data_procedures_nodups.shape
print(len(data_procedures) - len(data_procedures_nodups), "Duplikate entfernt.")

In [ ]:
# Gruppierung nach Patienten, Aufsummierung der Kosten
data_procedures_grouped = data_procedures_nodups.groupby(by="PATIENT")
data_procedures_final = data_procedures_grouped[["PRO_BASE_COST"]].sum(min_count=1)
data_procedures_final.rename(columns={"PRO_BASE_COST": "OVERALL_PRO_BASE_COST"}, inplace=True)
data_procedures_final["NUMBER_PROCEDURES"] = data_procedures_grouped[["PRO_BASE_COST"]].count()
data_procedures_final.shape

In [ ]:
data_procedures_final

## Erstellung Tabelle mit Gesamtkosten für Medikation pro Patient

In [ ]:
# Erstellung Subset mit Kostendaten zu Medikationen
data_medications = data_costs[["PATIENT", "ENC_ID", "MED_CODE", "MED_BASE_COST", "MED_DISPENSES", "MED_TOTAL_COST", "MED_PAYER_COVERAGE"]]
data_medications.shape

In [ ]:
# Entfernen von Duplikaten (gleicher Patient, gleicher Aufenthalt, gleiche Medikation, gleiche Kosten)
data_medications_nodups = data_medications.drop_duplicates()
data_medications_nodups.shape
print(len(data_medications) - len(data_medications_nodups), "Duplikate entfernt.")

In [ ]:
# Gruppierung nach Patienten, Aufsummierung der Kosten
data_medications_grouped = data_medications_nodups.groupby(by="PATIENT")
data_medications_final = data_medications_grouped[["MED_BASE_COST", "MED_DISPENSES", "MED_TOTAL_COST", "MED_PAYER_COVERAGE"]].sum(min_count=1)
data_medications_final.rename(columns={"MED_BASE_COST": "OVERALL_MED_BASE_COST", 
                                         "MED_DISPENSES": "OVERALL_MED_DISPENSES", 
                                         "MED_TOTAL_COST": "OVERALL_MED_TOTAL_COST", 
                                         "MED_PAYER_COVERAGE": "OVERALL_MED_PAYER_COVERAGE"}, inplace=True)
data_medications_final["NUMBER_MEDICATIONS"] = data_medications_grouped[["MED_BASE_COST"]].count()
data_medications_final.shape

In [ ]:
data_medications_final

## Erstellung einer Patiententabelle (mit demografischen Daten und Diagnosedaten)

In [ ]:
# Merge Informationen zu Patienten aus data_costs und Diagnosen aus data_diagnoses über Patienten-ID
patient_information = pd.merge(data_costs[["PATIENT", "PAT_GENDER", "PAT_RACE"]], data_diagnoses, how="inner", left_on=["PATIENT"], right_on=["patient"])
# Doppelte Zeilen entfernen
patient_information.drop_duplicates(inplace=True)
# Doppelte Spalte mit Patienten-ID entfernen
patient_information.drop(columns=["patient"], inplace=True)
# Spalte diagnosis umbenennen
patient_information.rename(columns={"diagnosis": "DIAGNOSIS"}, inplace=True)
# Spalte mit Patienten-ID als Index
patient_information.set_index("PATIENT", inplace=True)

patient_information.shape

In [ ]:
patient_information

## Erstellung der Analysetabelle als Kombination der zuvor erstellten Hilfstabellen


In [ ]:
# Merge alle Tabellen in einzelnen Dataframe über Patienten-ID (Index)
patient_data = pd.merge(pd.merge(pd.merge(patient_information, 
                                          data_encounters_final, left_index=True, right_index=True), 
                                 data_procedures_final, left_index=True, right_index=True), 
                        data_medications_final, left_index=True, right_index=True)
patient_data.reset_index(inplace=True)
patient_data.shape

In [ ]:
patient_data

In [ ]:
# Daten (Analysetabelle) in CSV Datei schreiben analysis_raw_data.csv
with open(os.path.join(DATA_PATH, "analysis_raw_data.csv"), 'w', encoding = 'utf-8') as f:
  patient_data.to_csv(f, index=False)

## Verifizierung der Analysetabelle mittels Checksum-Analyse

In [ ]:
# Hashwerte für Dataframe patient_data berechnen (ohne Spalte PATIENT)
patient_datahashes = hash_pandas_object(patient_data.drop(columns=['PATIENT']))

In [ ]:
# Checksumme der Hashwerte berechnen
patient_datahashes.sum()

In [ ]:
# Vergleich der berechneten Checksumme mit dem zu erwarteten Wert (4468229860989718260)
if patient_datahashes.sum() == 4468229860989718260:
  print("Die Überprüfung der Analysetabelle mittels Checksumme war erfolgreich. Die Checksumme stimmt mit dem im Projekt verwendeten Wert überein.")
else:
  print("Die Überprüfung der Analysetabelle mittels Checksumme war nicht erfolgreich. Es liegen Abweichungen zu den im Projekt verwendeten Daten vor.")

# EDA (Explorative Datenanalyse)

## Übersicht über Patientenanzahl (rows) und beschreibende Attribute (columns)

In [ ]:
patient_data.shape

## Übersicht über die beschreibenden Attribute (columns)

In [ ]:
patient_data.columns

## Übersicht über die vorhandenen Datentypen und Anzahl der Nullwerte

In [ ]:
patient_data.info()

## Erste Übersicht mit beschreibender Statistik (count, mean ...)

In [ ]:
patient_data.describe()

In [ ]:
patient_data.nunique(axis=0)

## Überprüfung auf Duplikate / doppelte Werte

In [ ]:
print("Anzahl doppelte Zeilen:", patient_data.duplicated(patient_data.columns).sum())

In [ ]:
# Entfernen von doppelten Werten (erster Wert bleibt bestehen)
patient_data.drop_duplicates(keep="first",inplace=True) 
print("Größe des Datensatzes nach dem Entfernen von doppelten Zeilen:", patient_data.shape)

## Überprüfung auf Vollständigkeit / fehlende Werte (NULL / NaN)

In [ ]:
# Anteil von Nicht-NULL-Werten in jeder Spalte
patient_data.count()/len(patient_data) * 100

In [ ]:
# Anzahl NULL-Werte pro Spalte
patient_data.isnull().sum()

# Datenanalyse

In [ ]:
# Erstellen einer Kopie des patient_data Dataframes für die Analyse
df = patient_data.copy()
df.head()

## Überprüfung der Daten mittels Hashes / Checksumme

Zu erwartender Wert: 4468229860989718260

In [ ]:
dfhashes = hash_pandas_object(df.drop(columns=['PATIENT']))

In [ ]:
dfhashes.sum()

In [ ]:
# Vergleich der berechneten Checksumme mit dem zu erwarteten Wert (4468229860989718260)
if dfhashes.sum() == 4468229860989718260:
  print("Die Überprüfung mittels Checksumme war erfolgreich. Der DataFrame df ist eine identische Kopie des DataFrames patient_data.")
else:
  print("Die Überprüfung mittels Checksumme war nicht erfolgreich.")

## Übersicht über die enthaltenen Patienten

### Anzahl der Patienten pro Diagnosegruppe (breast_cancer, other, none)

In [ ]:
# Erstellung eines Säulendiagramms zu den Diagnosen
plt_diagnosis = sns.countplot(x="DIAGNOSIS", data=df)

# Achsenbeschriftung und Titel ergänzen
plt_diagnosis.set(title='Patients by diagnosis group',
                  xlabel='Diagnosis group',
                  ylabel='Count')

# Beschriftung ergänzen
plt_diagnosis.bar_label(plt_diagnosis.containers[0], fmt='%.0f', label_type='edge', padding=1)

plt.savefig(os.path.join(RESULTS_PATH, "Plots_EDA", "01_patients_by_diagnosis.png"), dpi=300)

Bei den meisten Patienten wurde keine Diagnose gestellt (`none`). Bei 124 Patienten wurde eine Diagnose gestellt, die aber nicht Brustkrebs war (`other`). Nur bei einem Bruchteil der Patienten (n=11) ergab sich die Diagnose Brustkrebs (`breast_cancer`).

### Geschlechtsverteilung der Patienten

In [ ]:
# Erstellung eines Säulendiagramms für alle Patienten
plt_gender = sns.countplot(x="PAT_GENDER", data=df)

# Achsenbeschriftung und Titel ergänzen
plt_gender.set(title='Patient gender distribution',
               xlabel='Patient gender',
               ylabel='Count')

# Beschriftung ergänzen
plt_gender.bar_label(plt_gender.containers[0], fmt='%.0f', label_type='edge', padding=1)

plt.savefig(os.path.join(RESULTS_PATH, "Plots_EDA", "02_gender_distribution.png"), dpi=300)

Ausgeglichene Anzahl von weiblichen und männlichen Patienten.

In [ ]:
# Erstellung eines Säulendiagramms aufgesplittet nach Diagnose
plt_gender_diagnosis = sns.countplot(x="PAT_GENDER", hue="DIAGNOSIS", data=df)

# Achsenbeschriftung und Titel ergänzen
plt_gender_diagnosis.set(title='Patient gender by diagnosis group',
                         xlabel='Patient gender',
                         ylabel='Count')

for c in plt_gender_diagnosis.containers:
    # Beschriftung ergänzen
    plt_gender_diagnosis.bar_label(c, fmt='%.0f', label_type='edge', padding=1)

plt.savefig(os.path.join(RESULTS_PATH, "Plots_EDA", "03_gender_by_diagnosis.png"), dpi=300)

### Ethnischer Hintergrund der Patienten

In [ ]:
# Erstellung eines Säulendiagramms für alle Patienten
plt_ethnicity = sns.countplot(x="PAT_RACE", data=df)

# Achsenbeschriftung und Titel ergänzen
plt_ethnicity.set(title='Patient racial background',
                  xlabel='Patient racial background',
                  ylabel='Count')

# Beschriftung ergänzen
plt_ethnicity.bar_label(plt_ethnicity.containers[0], fmt='%.0f', label_type='edge', padding=1)

plt.savefig(os.path.join(RESULTS_PATH, "Plots_EDA", "04_ethnicity.png"), dpi=300)

Dieses Säulendiagramm zeigt, dass die meisten Patienten kaukasischer Abstammung waren. Andere Ethnien kamen nur in Ausnahmefällen vor. (Es müsste hier eine Normalisierung stattfinden, d.h. die gesamte Anzahl der Personen unterschiedlicher Ethnien müsste mit dem Patientenklientel verglichen werden.) 

In [ ]:
# Erstellung eines Säulendiagramms aufgesplittet nach Diagnose
plt_ethnicity_diagnosis = sns.countplot(x="PAT_RACE", hue="DIAGNOSIS", data=df)

# Achsenbeschriftung und Titel ergänzen
plt_ethnicity_diagnosis.set(title='Patient racial background by diagnosis group',
                            xlabel='Patient racial background',
                            ylabel='Count')

for c in plt_ethnicity_diagnosis.containers:
    # Beschriftung ergänzen
    plt_ethnicity_diagnosis.bar_label(c, fmt='%.0f', label_type='edge', padding=1)

plt.savefig(os.path.join(RESULTS_PATH, "Plots_EDA", "05_ethnicity_by_diagnosis.png"), dpi=300)

## Analyse der Kosten

### Ausgaben für Medikamente für die unterschiedlichen Diagnosen

In [ ]:
# Erstellung eines Scatter Plots für die Gesamtkosten der Medikation nach Diagnose
plt_costs_medication = df.plot(kind='scatter', x='DIAGNOSIS', y=('OVERALL_MED_TOTAL_COST'))

# Achsenbeschriftung und Titel ergänzen
plt_costs_medication.set(title='Costs for medication by diagnosis group',
                         xlabel='Diagnosis group',
                         ylabel='Total medication cost')

plt.savefig(os.path.join(RESULTS_PATH, "Plots_EDA", "06_medication_cost.png"), dpi=300)

Die Analyse der Ausgaben für Medikamente lassen keine Aussagen zu, um welche Diagnose es sich handelt.

### Berechnung der Kosten, die die Patienten mit den verschiedenen Diagnosen selbst tragen müssen.

In [ ]:
# Ersetzen von NaN Werten (bei Patienten ohne Prozeduren / Medikationen) mit 0 um Berechnungen zu ermöglichen
df = df.fillna(0)

Hier eine Aufstellung, wieviel die Medikamente insgesamt kosten und welcher Anteil von der Versicherung übernommen wird.

In [ ]:
df.groupby('DIAGNOSIS')['OVERALL_MED_TOTAL_COST', 'OVERALL_MED_PAYER_COVERAGE'].mean()

Hier eine Aufstellung, wieviel die Aufenthalte insgesamt kosten und wieviel von den Versicherungen übernommen wird.

In [ ]:
df.groupby('DIAGNOSIS')['OVERALL_ENC_BASE_COST', 'OVERALL_ENC_PAYER_COVERAGE'].mean()

Berechnung der Gesamtkosten, die der Patient selber tragen muss (für Medikamente und Aufenthalte)

In [ ]:
# Ergänzung einer Spalte Patient_Cost mit den Gesamtkosten pro Patient (exklusive der Prozeduren, für die im Datensatz keine Informationen zu Erstattungen vorliegen)
df.eval('PATIENT_COST = (OVERALL_ENC_BASE_COST - OVERALL_ENC_PAYER_COVERAGE) + (OVERALL_MED_TOTAL_COST - OVERALL_MED_PAYER_COVERAGE)', inplace = True)
# Ergänzung einer Spalte Patient_Cost_Pro mit den Gesamtkosten pro Patient (mit laut Datensatz nicht erstatteten Prozeduren)
df.eval('PATIENT_COST_PRO = (OVERALL_ENC_BASE_COST - OVERALL_ENC_PAYER_COVERAGE) + (OVERALL_MED_TOTAL_COST - OVERALL_MED_PAYER_COVERAGE) + OVERALL_PRO_BASE_COST', inplace = True)
df.columns

In [ ]:
# Daten (erweiterte Analysetabelle) in CSV Datei schreiben analysis_processed_data.csv
with open(os.path.join(DATA_PATH, "analysis_processed_data.csv"), 'w', encoding = 'utf-8') as f:
  patient_data.to_csv(f, index=False)

In [ ]:
# Gruppierung der erweiterten Analysetabelle nach Diagnosegruppe
df_grouped = df.groupby('DIAGNOSIS')

In [ ]:
# Berechnung der durchschnittlichen Kosten / Erstattungen pro Diagnosegruppe
df_grouped.mean().fillna(0)

In [ ]:
# Erstellung eines Dataframes der Gesamtkosten pro Patient und der entsprechenden Diagnosegruppe
df_cost = df[['PATIENT_COST', 'DIAGNOSIS']]

In [ ]:
df_cost.head()

In [ ]:
# Gruppierung des erstellten Dataframes nach Diagnosegruppe
df_cost_grouped = df_cost.groupby('DIAGNOSIS')

In [ ]:
df_cost_grouped.mean()

In [ ]:
# Erstellung eines Säulendiagramms der Kosten aufgesplittet nach Diagnose
plt_costs_diagnosis = sns.barplot(data=df_cost, x="DIAGNOSIS", y="PATIENT_COST", ci=None)

# Achsenbeschriftung und Titel ergänzen
plt_costs_diagnosis.set(title='Healthcare costs by diagnosis group (excluding procedures)',
                        xlabel='Diagnosis group',
                        ylabel='Average cost per patient')

plt_costs_diagnosis.bar_label(plt_costs_diagnosis.containers[0], fmt='%.0f', label_type='edge', padding=1)

plt.savefig(os.path.join(RESULTS_PATH, "Plots_EDA", "07_healthcare_cost_by_diagnosis.png"), dpi=300)

Die Analyse zeigt, dass Patienten mit Brutkrebs im Durchschnitt etwas mehr Kosten selber tragen müssen, als Patienten mit einer anderen Diagnose oder gar keiner Diagnose. Die niedrigen Kosten für Patienten ohne eine gegebene Diagnose sind darauf zurückzuführen, dass diese keine Medikationen verschrieben bekommen haben.

In [ ]:
# Erstellung eines Dataframes der Gesamtkosten pro Patient, inklusive Prozeduren, und der entsprechenden Diagnosegruppe
df_cost_pro = df[['PATIENT_COST_PRO', 'DIAGNOSIS']]
df_cost_pro.head()

In [ ]:
# Gruppierung des erstellten Dataframes nach Diagnosegruppe
df_cost_pro_grouped = df_cost_pro.groupby('DIAGNOSIS')
df_cost_pro_grouped.mean()

In [ ]:
# Erstellung eines Säulendiagramms der Kosten (inklusive Prozeduren) aufgesplittet nach Diagnose
plt_costs_pro_diagnosis = sns.barplot(data=df_cost_pro, x="DIAGNOSIS", y="PATIENT_COST_PRO", ci=None)

# Achsenbeschriftung und Titel ergänzen
plt_costs_pro_diagnosis.set(title='Healthcare costs by diagnosis group (including procedures)',
                            xlabel='Diagnosis group',
                            ylabel='Average cost per patient')

plt_costs_pro_diagnosis.bar_label(plt_costs_pro_diagnosis.containers[0], fmt='%.0f', label_type='edge', padding=1)

plt.savefig(os.path.join(RESULTS_PATH, "Plots_EDA", "08_healthcare_cost_pro_by_diagnosis.png"), dpi=300)

Unter Einbeziehung der Kosten für Prozeduren (die aufgrund der fehlenden Informationen zu Erstattungen komplett den Patienten zugeordnet wurden) ergibt sich ein signifikanter Unterschied zwischen Patienten mit Brustkrebs und Patienten mit anderen Diagnosen, mit mer als doppelt so hohen Kosten. 

In [ ]:
df_encounters = df[['DIAGNOSIS', 'NUMBER_ENCOUNTERS']]

In [ ]:
df_encounters.head()

In [ ]:
df_encounters_grouped = df_encounters.groupby('DIAGNOSIS')

In [ ]:
df_encounters_grouped.mean()

In [ ]:
# Erstellung eines Säulendiagramms der Aufenthalte aufgesplittet nach Diagnose
plt_encounters_diagnosis = sns.barplot(data=df_encounters, x="DIAGNOSIS", y="NUMBER_ENCOUNTERS", ci=None)

plt_encounters_diagnosis.set(title='Healthcare encounters by diagnosis group',
                             xlabel='Diagnosis group',
                             ylabel='Average number of encountres per patient')

plt_encounters_diagnosis.bar_label(plt_encounters_diagnosis.containers[0], fmt='%.0f', label_type='edge', padding=1)

plt.savefig(os.path.join(RESULTS_PATH, "Plots_EDA", "09_healthcare_encounters_by_diagnosis.png"), dpi=300)

Die Analyse zeigt, dass Patienten mit Brustkrebs deutlich öfter den Arzt/ein Krankenhaus aufsuchen als Patienten mit einer anderen oder gar keiner Diagnose.

# Machine Learning

Ziel: Anhand der Kosten, die für den Patienten entstehen, und der Anzahl der Arztbesuche soll auf die Diagnose der einzelnen Patienten geschlossen werden.

In [ ]:
# Erstellung eines Ausschnitts der Analysetabelle mit den erforderlichen Spalten
df_ml = df[['DIAGNOSIS', 'NUMBER_ENCOUNTERS', 'PATIENT_COST']]
df_ml.head()

In [ ]:
# Gruppierung des Dataframes nach Diagnosegruppe
df_ml.groupby('DIAGNOSIS')
df_ml.groupby('DIAGNOSIS').head()

In [ ]:
# Entfernen von NaNs
df_rest = df_ml.dropna(axis=0)
df_rest.head()

## 1.Decision Tree Modell

Aufgrund der geringen Anzahl an vollständigen Datensätzen wird zunächst ein Decision Tree Modell versucht

In [ ]:
# Import der benötigten Libraries
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

# Warnungen ausblenden
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Definition von X / y
features = df_rest.columns.tolist()
features.remove('DIAGNOSIS')
X = df_rest[features]
y = df_rest.DIAGNOSIS

# Aufsplitten des Datensatzes in Trainings- und Testdatensatz
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=150, shuffle=True)

# Trainings- und Testdatensätze überprüfen
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

Es stehen 713 vollständige Trainingsdatensätze und 306 vollständige Testdatensätze zur Verfügung.

In [ ]:
# Training eines Decision Trees mit dem gini-Index
# Erstellung des Classifier-Objekts
clf_gini = DecisionTreeClassifier(criterion = "gini", 
            max_depth=3, min_samples_leaf=5) 
# Durchführung des Trainings
clf_gini.fit(X_train, y_train)

In [ ]:
# Vorhersage der Diagnose anhand der Testdaten (mit giniIndex)
y_pred = clf_gini.predict(X_test) 
print("Predicted values:\n") 
print(y_pred)

In [ ]:
# Darstellung der Konfusionsmatrix und der Accuracy
print("-----"*15)
print("Confusion Matrix: \n", 
confusion_matrix(y_test, y_pred)) 

print("-----"*15)
print ("Accuracy : \n", 
accuracy_score(y_test,y_pred)*100) 
    
print("-----"*15)
print("Report : \n", 
classification_report(y_test, y_pred))

In [ ]:
# Darstellung und Evaluation mithilfe einer Konfusionsmatrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, cmap='YlGnBu', annot=True, fmt='d', linewidths=.5)

plt.savefig(os.path.join(RESULTS_PATH, "Plots_MachineLearning", "01_decision_tree_cm.png"), dpi=300)

Im vorliegenden Modell wurden die Patienten mit Brustkrebs nicht erkannt. Eine Erklärung dafür ist, dass sich alle Patienten mit Brustkrebs im Testset befanden. Da keine Brustkrebs-Patienten im Trainingsset vorhanden waren, konnte das Modell diese Diagnose nicht "lernen" und hat sie daher auch nicht erkannt.

## 2. Logistic Regression Analyse

In [ ]:
# Import der benötigten Libraries
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Definition des Plotting-Styles
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (11, 7)

In [ ]:
# Logistische Regression (Verwendung des vorher definierten Trainings- / Testdatensatzes)
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred_train = log_reg.predict(X_train)
y_pred = log_reg.predict(X_test)

In [ ]:
# Bestimmung der Accuracy
print("Accuracy on train set:", round(accuracy_score(y_train, y_pred_train), 2))
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred), 2))
print("--------"*10)

In [ ]:
# Ausgabe des Classification Report
print(classification_report(y_test, y_pred))
print("--------"*10)

In [ ]:
# Darstellung und Evaluation mithilfe einer Konfusionsmatrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, cmap='YlGnBu', annot=True, fmt='d', linewidths=.5);

plt.savefig(os.path.join(RESULTS_PATH, "Plots_MachineLearning", "02_logistic_regression_analysis_cm.png"), dpi=300)

Mit diesem Ansatz wurden 3 der 4 Brustkrebspatienten erkannt. Das Modell konnte die Diagnose mithilfe des Trainingssets lernen. Die Aufteilung in Trainings- und Testset war in diesem Ansatz ausgeglichener.

Aufgrund der geringen Fallzahlen sind die Ergebnisse dieser Modelle nicht belastbar. Für eine aussagekräftige Analyse ist es notwendig das Patientenkollektiv deutlich zu vergrößern.